In [2]:
import networkx as nx
import graph_tools
import graph_viz


/opt/anaconda3/envs/thesis/lib/python3.12/site-packages/dash_bootstrap_components/_table.py:5: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [12]:
graph = nx.DiGraph()

graph.add_edge("A", "B", weight=1)
graph.add_edge("B", "C", weight=1)
graph.add_edge("C", "A", weight=1)
graph.add_edge("D", "A", weight=0.5)
graph.add_edge("D", "E", weight=0.5)

In [22]:
graph = nx.DiGraph()

graph.add_edge('A', 'B', weight=1)
graph.add_edge('B', 'C', weight=0.5)
graph.add_edge('C', 'A', weight=1)
graph.add_edge('B', 'E', weight=0.5)
graph.add_edge('E', 'C', weight=0.5)
graph.add_edge('E', 'A', weight=0.5)

graph.add_edge('X', 'Z', weight=1)
graph.add_edge('Y', 'Z', weight=1)
graph.add_edge('Z', 'A', weight=0.5)
graph.add_edge('Z', 'W', weight=0.5)

graph.add_edge('M', 'M', weight=1)
graph.add_edge('N', 'M', weight=0.5)
graph.add_edge('N', 'N', weight=0.5)

In [23]:
def collapse_terminal_sccs(graph, lost_node_name="lost"):
    """
    Replaces terminal strongly connected components with a single 'lost' sink node.
    All external delegations into the SCC are redirected to the lost node.
    """
    
    changed = False
    sccs = list(nx.strongly_connected_components(graph))

    for scc in sccs:
        if lost_node_name in scc:
            continue  # skip lost node if it was already added earlier

        # Check if SCC is terminal
        is_terminal = True
        for node in scc:
            if graph.out_degree(node) == 0:
                is_terminal = False  # it's a sink, so SCC is not terminal
                break
            
            for _, v in graph.out_edges(node):
                if v not in scc:
                    is_terminal = False
                    break
            if not is_terminal:
                break

        if is_terminal:
            # Redirect incoming edges to SCC from outside
            for node in scc:
                for u, _ in list(graph.in_edges(node)):
                    if u not in scc:
                        weight = graph[u][node].get("weight")
                        if graph.has_edge(u, lost_node_name):
                            graph[u][lost_node_name]["weight"] += weight
                        else:
                            graph.add_edge(u, lost_node_name, weight=weight)
                        graph.remove_edge(u, node)

            # Remove the entire SCC
            graph.remove_nodes_from(scc)
            changed = True
            break  # restart since the graph changed

    return changed

# Keep removing terminal SCCs until none are left
while collapse_terminal_sccs(graph):
    pass


In [24]:
list(graph.edges.data('weight'))

[('X', 'Z', 1),
 ('Z', 'W', 0.5),
 ('Z', 'lost', 0.5),
 ('Y', 'Z', 1),
 ('N', 'N', 0.5),
 ('N', 'lost', 0.5)]